## Inpatient activity

### Mitigation data

We have the table `nhp.raw_data.apc_mitigators` which we will use for collecting data on mitigation. However, this goes to the granularity of 92 mitigators, whilst we are concerning ourselves with three *types* of mitigation, as well as the counterfactual: re-direction and substitution ("reSu"), prevention ("prev") and de-adoption ("deAdopt"). There are also instances in which a specific hospital admission is in scope of more than mitigator; in this case we choose assign the admission according to a hierachy: prevention > de-adoption > re-direction and substitution; else "none". Finally, the mitigators have rates which we must sample from to reflect the fact that now all cases meeting the [definitions for each mitigator](https://connect.strategyunitwm.nhs.uk/nhp/project_information/modelling_methodology/activity_mitigators/inpatient_activity_mitigators.html) will actually be relevant.

The below query creates a view `activity_mitigation_table` where the sampling, grouping and hierachy have been effected.

In [0]:
from pyspark.sql.functions import col, when, rand

# Load the data from the table
df = spark.table("nhp.raw_data.apc_mitigators")

# load the apc records (needed for filtering on group / pod)
apc_pod = spark.table("nhp.raw_data.apc").select("epikey", "fyear", "provider", "group")

# Load the diagnoses (as we need to filter out obesity strategies where the principal diagnosis is I12 or I22)
diagnoses = spark.table("hes.silver.apc_diagnoses").filter(col("diag_order") == 1).select("epikey", "fyear", "diagnosis")

# Apply transformations
df_transformed = (
    df.withColumn(
        "mitigation_type",
        when(
            col("strategy").isin(
                'alcohol_partially_attributable_acute', 'alcohol_partially_attributable_chronic', 'alcohol_wholly_attributable', 'falls_related_admissions',
                'obesity_related_admissions', 'smoking'
            ), 'prev'
        ).when(
            col("strategy").isin(
                'evidence_based_interventions_ent', 'evidence_based_interventions_general_surgery', 'evidence_based_interventions_gi_surgical',
                'evidence_based_interventions_msk', 'evidence_based_interventions_urology', 'evidence_based_interventions_vascular_varicose_veins'
            ), 'deAdopt'
        ).when(
            col("strategy").isin(
                'ambulatory_care_conditions_acute', 'ambulatory_care_conditions_chronic', 'ambulatory_care_conditions_vaccine_preventable',
                'eol_care_2_days', 'eol_care_3_to_14_days', 'frail_elderly_high', 'frail_elderly_intermediate', 'intentional_self_harm',
                'medically_unexplained_related_admissions', 'medicines_related_admissions_explicit', 'medicines_related_admissions_implicit_anti-diabetics',
                'medicines_related_admissions_implicit_benzodiasepines', 'medicines_related_admissions_implicit_diurectics',
                'medicines_related_admissions_implicit_nsaids', 'raid_ae', 'readmission_within_28_days', 'zero_los_no_procedure_adult',
                'zero_los_no_procedure_child'
            ), 'reSu'
        ).otherwise('none')
    ).filter(col("sample_rate") >= rand())
    # join to diagnoses in order to filter out specific diagnoses for obesity
    .join(diagnoses, ["epikey", "fyear"], "left")
    .filter(~((col("strategy") == "obesity_related_admissions") & (col("diagnosis").rlike("^I[12]2"))))
    # join to apc data to get the group/pod column for filtering out elective reSu and non-elective de-adoption
    .join(apc_pod, ["epikey", "provider", "fyear"])
    .filter(~((col("group") == "elective") & (col("mitigation_type") == "reSu")))
    .filter(~((col("group") == "non-elective") & (col("mitigation_type") == "deAdopt")))
    .drop("group", "diagnosis")
)

# Create or replace the table
df_transformed.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("apc_mitigation_data_unranked")

In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F


# Load the data
apc_mitigation_data_unranked = spark.table("strategyunit.default.apc_mitigation_data_unranked")

# Define the window specification
windowSpec = Window.partitionBy("epikey").orderBy(
    F.when(F.col("mitigation_type") == "prev", 1)
    .when(F.col("mitigation_type") == "deAdopt", 2)
    .when(F.col("mitigation_type") == "reSu", 3)
    .otherwise(4)
)

# Add the row number column
ranked_data = apc_mitigation_data_unranked.withColumn("rn", F.row_number().over(windowSpec))

# Filter the rows where rn = 1
result = ranked_data.filter(F.col("rn") == 1)

# Create or replace the table
result.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("apc_mitigation_data_ranked")

### Note on missing mitigators
In the query above we 30 specific strategies that have been allocated to one of the three types of mitigation. 

However, there are 33 inpatient activity avoidance mitigators (derived below), meaning there are three missing:

- _cancelled_operations_ - this is more of an efficiency strategy
- _virtual_wards_activity_avoidance_ari_ - this is new and still being tested
- _virtual_wards_activity_avoidance_heart_failure_ - this is new and still being tested


### ICD10 chapters

One of the things we want the data by is ICD10 chapter of the principal diagnosis.

There is an existing reference table `Su_data.reference.icd10_codes` but we just need to create a new copy `icd10_codes_gabriel` as the main refence table does not have the chapters correctedly tabulated.

In [0]:
%sql
CREATE OR REPLACE VIEW icd10_codes_gabriel as
SELECT
    *,
    CASE 
        WHEN chapter_description = 'Certain infectious and parasitic diseases' THEN 'I'
        WHEN chapter_description = 'Neoplasms' THEN 'II'
        WHEN chapter_description = 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism' THEN 'III'
        WHEN chapter_description = 'Endocrine, nutritional and metabolic diseases' THEN 'IV'
        WHEN chapter_description = 'Mental and behavioural disorders' THEN 'V'
        WHEN chapter_description = 'Diseases of the nervous system' THEN 'VI'
        WHEN chapter_description = 'Diseases of the eye and adnexa' THEN 'VII'
        WHEN chapter_description = 'Diseases of the ear and mastoid process' THEN 'VIII'
        WHEN chapter_description = 'Diseases of the circulatory system' THEN 'IX'
        WHEN chapter_description = 'Diseases of the respiratory system' THEN 'X'
        WHEN chapter_description = 'Diseases of the digestive system' THEN 'XI'
        WHEN chapter_description = 'Diseases of the skin and subcutaneous tissue' THEN 'XII'
        WHEN chapter_description = 'Diseases of the musculoskeletal system and connective tissue' THEN 'XIII'
        WHEN chapter_description = 'Diseases of the genitourinary system' THEN 'XIV'
        WHEN chapter_description = 'Pregnancy, childbirth and the puerperium' THEN 'XV'
        WHEN chapter_description = 'Certain conditions originating in the perinatal period' THEN 'XVI'
        WHEN chapter_description = 'Congenital malformations, deformations and chromosomal abnormalities' THEN 'XVII'
        WHEN chapter_description = 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified' THEN 'XVIII'
        WHEN chapter_description = 'Injury, poisoning and certain other consequences of external causes' THEN 'XIX'
        WHEN chapter_description = 'External causes of morbidity and mortality' THEN 'XX'
        WHEN chapter_description = 'Factors influencing health status and contact with health services' THEN 'XXI'
        WHEN chapter_description = 'Codes for special purposes' THEN 'XXII'
    END AS chapter_number
FROM strategyunit.reference.icd10_codes 

### Extracting data

The aim is to extract counts of activity (spells) by the following groups:

-   financial year
-   admission type / pod
-   age
-   sex
-   region
-   ICD-10 chapter
-   deprivation quintile
-   mitigation type

In [0]:
from pyspark.sql import functions as F

apc = spark.table("nhp.raw_data.apc")
apc_mitigation_data = spark.table("apc_mitigation_data_ranked")
icd_10_chapter = spark.table("icd10_codes_gabriel")

apc_results = (
    apc
    .filter((F.col("apc.fyear") <= 201920) & (F.col("apc.fyear") >= 201112))
    .filter(~(F.col("group") == "maternity"))
    .join(apc_mitigation_data, "epikey", "left")
    .join(icd_10_chapter, apc["primary_diagnosis"] == icd_10_chapter["icd10"], "left")
    .withColumn("mitigation_type", F.coalesce(F.col("mitigation_type"), F.lit("none")))
    .groupBy(F.col("apc.fyear"), apc["group"].alias("pod"), "age", "resgor_ons", "sex", "chapter_number", "imd_quintile", "mitigation_type")
    .agg(F.count("epikey").alias("activity"))
    .dropna(subset=["fyear", "age", "sex", "imd_quintile", "resgor_ons"])
)

display(apc_results)

## Outpatients

Consulting the lookup there are 16 outpatients. Each of these will be 4 variants, so in reality it's 4 OPA groupings.

These are the ones in Stevem's lookup:
- consultant_to_consultant_reduction_adult_non-surgical
- consultant_to_consultant_reduction_adult_surgical
- consultant_to_consultant_reduction_child_non-surgical
- consultant_to_consultant_reduction_child_surgical
- followup_reduction_adult_non-surgical
- followup_reduction_adult_surgical
- followup_reduction_child_non-surgical
- followup_reduction_child_surgical


We're off by one set of four:
- gp_referred_first_attendance_reduction_adult_non-surgical
- gp_referred_first_attendance_reduction_adult_surgical
- gp_referred_first_attendance_reduction_child_non-surgical
- gp_referred_first_attendance_reduction_child_surgical

There are the following four but we consider these to be out of scope as they are efficiency strategies:
- convert_to_tele_adult_non-surgical
- convert_to_tele_adult_surgical
- convert_to_tele_child_non-surgical
- convert_to_tele_child_surgical





### Extraction

We create a table `mitigator_flags` that removes any outpatient strategies in the Convert to Tele-attendance group and also to remove those which are relating to maternity, since these do not exist in the mitigator lookup.

This is  breaking down due to the large amount of data in the outpatients table so we split the data into three extracts: one for 2011-12 to 2014-15, another for 2015-16 to 2018-19, and a final one for 2019-20. 

In [0]:
from pyspark.sql import functions as F

# Load the opa_mitigators table
opa_mitigators = spark.table("nhp.raw_data.opa_mitigators")

# Create the mitigator_flags DataFrame
mitigator_flags = (
    opa_mitigators
    .filter(
        (~opa_mitigators.strategy.like("convert_to_tele%")) & 
        (~opa_mitigators.strategy.like("%maternity")) 
    )
    .groupBy("attendkey", "provider", "fyear")
    .agg(
        F.max(F.when(opa_mitigators.attendkey.isNotNull(), 1).otherwise(0)).alias("has_mitigation")
    )
)

# Load the opa table
opa = spark.table("nhp.raw_data.opa")

# Join opa with mitigator_flags
result = opa.join(mitigator_flags, on=["attendkey", "provider", "fyear"], how="left")

# Add the mitigation_type column
result = result.withColumn(
    "mitigation_type",
    F.when(result.has_mitigation == 1, "deAdopt").otherwise("none")
)

# Remove prefixes "adult_" or "child_" from the type column
result = result.withColumn(
    "type",
    F.regexp_replace(result["type"], "^(adult_|child_)", "")
)

# drop NA in the fields being used for joining to population data
result = result.dropna(subset=["fyear", "age", "sex", "imd_quintile", "resgor_ons"])

# Group by the required columns and count distinct attendkey
opa_data = result.filter((result.fyear >= 201112) & (result.fyear <= 201920)).groupBy(
    F.lit("opa").alias("pod"),
    "fyear",
    "age",
    "sex",
    "type",
    "imd_quintile",
    "resgor_ons",
    "mitigation_type"
).agg(
    F.countDistinct("attendkey").alias("activity")
)

# Create or replace a view instead of saving as a table
opa_data.createOrReplaceTempView("opa_mitig_data")

In [0]:
%sql
select * from opa_mitig_data where fyear between 201112 and 201415;

In [0]:
%sql
select * from opa_mitig_data where fyear between 201516 and 201819;

In [0]:
%sql
select * from opa_mitig_data where fyear = 201920;

## A&E activity

### Categorising the mitigators for ED 

In the below chunk we create a temporary view which 

1. populates the region variable in 201920 using the region lookup on the `resladst_ons` variable (given the existing `resgor_ons` variable is full of NAs) and
2. marks a row as "reSu" if it meets any one of the four conditions for this (frequent attenders, left before treatment, low cost referred or discharged, discharged no treatment)
3. removes any NAs on the joining columns

In [0]:
ecds_data = spark.table("nhp.raw_data.ecds")
lookup = spark.table("strategyunit.default.lad_19_rgn_19_en_lu_948748_b_0_eaa_54_fe_888_a_604_b_126_f_5_e_672_4818463568216355194").select("LAD19CD", "RGN19CD")

# Filter for aedepttype == "01" and fyear between 201112 and 201920
ecds_filtered = ecds_data.filter(
    (F.col("aedepttype") == "01") & 
    (F.col("fyear").between(201112, 201920))
)

# Overwrite resgor_ons and drop LAD19CD and RGN19CD columns
ecds_mitig_view = ecds_filtered.join(lookup, ecds_filtered.resladst_ons == lookup.LAD19CD, "left").withColumn(
    "resgor_ons",
    F.when(F.col("fyear") < 201920, F.col("resgor_ons")).otherwise(F.col("RGN19CD"))
).drop("LAD19CD", "RGN19CD")

ecds_mitig_view = ecds_mitig_view.withColumn(
    "mitigation_type",
    F.when(
        (F.col("is_frequent_attender") == 1) |
        (F.col("is_left_before_treatment") == 1) |
        (F.col("is_low_cost_referred_or_discharged") == 1) |
        (F.col("is_discharged_no_treatment") == 1),
        "reSu"
    ).otherwise("none")
).dropna(subset=["fyear", "age", "sex", "imd_quintile", "resgor_ons"])

ecds_mitig_view.createOrReplaceTempView("ecds_mitig_view")

### Extracting the ED data

Now we can simply extract the data

In [0]:
aae_mitig_data = ecds_mitig_view.groupBy(
    F.lit("aae").alias("pod"),
    "fyear",
    "age",
    "sex",
    "group",
    "imd_quintile",
    "resgor_ons",
    "mitigation_type"
).agg(
    F.count("key").alias("activity")
)


In [0]:
aae_mitig_data.display()

## Table showing strategies in base year and final year by pod

We want a table which presents the activity in the baseline of 2011/12 and the final year 2019/20 cut by activity subset. Activity subsets are a slight aggregation of the strategies in the mitigation table here -- for example, "alcohol_partially_attributable_acute", "alcohol_partially_attributable_chronic", "alcohol_wholly_attributable" are grouped together under "Alcohol Related Admissions". 

For admitted patient care, this will represent the data *before* the individual admissions are assigned to only one mitigation type (i.e. a many:1 relationship rather than many:many), and *after* the sampling of attributable fractions is conducted (i.e. not every single alcohol partial admission is expected to be relevant).

For opa and aae, there is no issue with assigning a specific mitigation type or sampling. We simply count the specific strategies.

For all, we include subtotals for non-mitigatable and mitigatable. These two will sum to the total activity. The breakdown of mitigatable activity in activity subsets is *not* expected to equal the subtotal given that specific admissions / attendances may be in scope of more than one strategy / activity subset.


### Conversion to activity subset

We must do this aggregation in Data Bricks as, at least for inpatient data, there are episodes which appear in more than one strategy under the same subset. We can see this in the below table which shows individual spells appearing in more than one of the strategies under "Alcohol Related Admissions".

In [0]:
from pyspark.sql.functions import count, col

# Load the table into a DataFrame
apc_mitigators_df = spark.table("apc_mitigation_data_ranked")

# Filter the DataFrame for the specified strategies
filtered_df = apc_mitigators_df.filter(
    apc_mitigators_df.strategy.isin(
        "alcohol_wholly_attributable", 
        "alcohol_partially_attributable_acute", 
        "alcohol_partially_attributable_chronic"
    )
)

# Filter for epikeys that appear more than once
epikey_counts = filtered_df.groupBy("epikey").agg(count("*").alias("count"))
repeated_epikeys = epikey_counts.filter(col("count") > 1).select("epikey")

filtered_df = filtered_df.join(repeated_epikeys, on="epikey", how="inner")

filtered_df.display()

### Extraction of data

In [0]:
%python
from pyspark.sql.functions import col, when, count, countDistinct, lit

## Load tables
apc = spark.read.table("nhp.raw_data.apc")
apc_mitigation_data_unranked = spark.read.table("apc_mitigation_data_unranked")
opa = spark.read.table("nhp.raw_data.opa")
opa_mitigators = spark.read.table("nhp.raw_data.opa_mitigators")
ecds = spark.read.table("nhp.raw_data.ecds")
aae_mitigators_data = spark.read.table("aae_mitigators_data")
activity_subset_lookup = spark.read.table("strategyunit.default.activity_subset_lookup")

# Join with activity subset lookup
apc_subset_data = apc_mitigation_data_unranked.join(activity_subset_lookup, on="strategy", how="inner")
opa_subset_data = opa_mitigators.join(activity_subset_lookup, on="strategy", how="inner")
aae_subset_data = aae_mitigators_data.join(activity_subset_lookup, on="strategy", how="inner")

# apc
apc_subset_table = apc.join(
    apc_subset_data, 
    [apc.epikey == apc_subset_data.epikey, apc.fyear == apc_subset_data.fyear], 
    "left"
).dropna(
    subset=["apc.fyear", "age", "sex", "imd_quintile", "resgor_ons"]
).filter(
    apc.fyear.isin(201112, 201920)
).groupBy(
    apc.fyear,
    apc.group.alias("pod"),
    when(apc_subset_data.mitigation_type.isNull(), 'none')
    .otherwise(apc_subset_data.mitigation_type).alias("mitigation_type"),
    when(apc_subset_data.mitigation_type.isNull(), 'none')
    .otherwise(apc_subset_data.activity_subset).alias("activity_subset")
).agg(
    count(apc.epikey).alias("activity")
)


## opa
opa_subset_table = opa.join(
    opa_subset_data,
    (opa.attendkey == opa_subset_data.attendkey) & 
    (opa.provider == opa_subset_data.provider) & 
    (opa.fyear == opa_subset_data.fyear),
    "left"
).dropna(
    subset=["opa.fyear", "age", "sex", "imd_quintile", "resgor_ons"]
).filter(
    opa.fyear.isin(201112, 201920)
).withColumn(
    "pod", lit("opa")
).groupBy(
    opa.fyear,
    col("pod"),
    when(opa_subset_data.strategy.isNull(), 'none')
    .otherwise('deAdopt').alias("mitigation_type"),
    when(opa_subset_data.activity_subset.isNull(), 'none')
    .otherwise(opa_subset_data.activity_subset).alias("activity_subset")
).agg(
    count(opa.attendkey).alias("activity")
)

## aae
# Filter and label subsets
frequent_attenders = ecds_mitig_view.filter(col("is_frequent_attender") == 1).select(
    "key", lit("A&E Frequent Attenders").alias("activity_subset")
)

minimal_clinical_input = ecds_mitig_view.filter(
    (col("is_discharged_no_treatment") == 1) | 
    (col("is_low_cost_referred_or_discharged") == 1)
).select(
    "key", lit("A&E Attendances requiring minimal clinical input").alias("activity_subset")
)

left_before_treatment = ecds_mitig_view.filter(col("is_left_before_treatment") == 1).select(
    "key", lit("A&E Patients Left Before Being Treated").alias("activity_subset")
)

# Union all subsets
aae_mitig_key_lookup = frequent_attenders.unionByName(minimal_clinical_input).unionByName(left_before_treatment)

# Join back to full data and aggregate
aae_subset_table = (
    ecds_mitig_view.join(aae_mitig_key_lookup, "key", "left")
    .filter(col("fyear").isin(201112, 201920))
    .withColumn("pod", lit("aae"))
    .groupBy(
        col("fyear"),
        col("pod"),
        when(col("activity_subset").isNull(), "none")
            .otherwise("reSu").alias("mitigation_type"),
        when(col("activity_subset").isNull(), "none")
            .otherwise(col("activity_subset")).alias("activity_subset")
    )
    .agg(count("key").alias("activity"))
)

# Union all parts
result = apc_subset_table.unionByName(opa_subset_table).unionByName(aae_subset_table)
result.display()

## Previous SQL queries on Strategy Unit Server

We have the following three scripts for categorising strategies from the previous analysis



### in-patient prevention

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_prevention


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_prevention

from [HESData].[nhp_strategies].[ip_alcohol_partially_attributable] 
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_alcohol_wholly_attributable]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_falls_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_obesity_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_smoking]
where fraction >= RAND(CHECKSUM(NewId()))

```

### in-patient re-direction and substitution

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_redirect_substitute


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_redirect_substitute

from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_acute]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_chronic]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_vaccine_preventable]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_eol_care]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_frail_elderly]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_intentional_self_harm]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medically_unexplained_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_explicit]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_anti_diabetics]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_benzodiasepines]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_diurectics]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_nsaids]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_raid_ae]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_readmission_within_28_days]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_zero_los_no_procedure]
where fraction >= RAND(CHECKSUM(NewId()))
```



### in-patient de-adoption

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_rationing


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_rationing

from [HESData].[nhp_strategies].[ip_evidence_based_interventions_ent]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_general_surgery]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_gi_surgical]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_msk]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_urology]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_vasuclar_varicose_veins]
where fraction >= RAND(CHECKSUM(NewId()))
```



### outpatient de-adoption
```sql
use StrategicWorking

--drop table dbo.sw_op_mitigatable_rationing

select attendkey, fyear

into dbo.sw_op_mitigatable_rationing

from [HESData].[nhp_strategies].[op_followup_reduction]
where fraction >= RAND(CHECKSUM(NewId()))

union


select attendkey, fyear
from [HESData].[nhp_strategies].[op_consultant_to_consultant_referrals]
where fraction >= RAND(CHECKSUM(NewId()))
```

### emergency department re-direction and substitution
```sql
use StrategicWorking

--drop table dbo.sw_ed_mitigatable_redirect_substitute


select aekey, fyear

into dbo.sw_ed_mitigatable_redirect_substitute

from [HESData].[nhp_strategies].[aae_frequent_attender]
where fraction >= RAND(CHECKSUM(NewId()))

union


select aekey, fyear
from [HESData].[nhp_strategies].[aae_left_before_treatment]
where fraction >= RAND(CHECKSUM(NewId()))


union


select aekey, fyear
from [HESData].[nhp_strategies].[aae_low_cost_discharged]
where fraction >= RAND(CHECKSUM(NewId()))
```